**CASE 1** 

In [70]:
import numpy as np
import pandas as pd
from scipy import stats

In [71]:
# Ratings
ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D']
rating_to_idx = {r: i for i, r in enumerate(ratings)}

# Transition matrix (probabilities)
transition_matrix = np.array([
    [0.91115, 0.08179, 0.00607, 0.00072, 0.00024, 0.00003, 0.00000, 0.00000],  # AAA
    [0.00844, 0.89626, 0.08954, 0.00437, 0.00064, 0.00036, 0.00018, 0.00021],  # AA
    [0.00055, 0.02595, 0.91138, 0.05509, 0.00499, 0.00107, 0.00045, 0.00052],  # A
    [0.00031, 0.00147, 0.04289, 0.90584, 0.03898, 0.00708, 0.00175, 0.00168],  # BBB
    [0.00007, 0.00044, 0.00446, 0.06741, 0.83274, 0.07667, 0.00895, 0.00926],  # BB
    [0.00008, 0.00031, 0.00150, 0.00490, 0.05373, 0.82531, 0.07894, 0.03523],  # B
    [0.00000, 0.00015, 0.00023, 0.00091, 0.00388, 0.07630, 0.83035, 0.08818],  # CCC
])

# Forward bond values at t=1
forward_values = np.array([99.50, 98.51, 97.53, 92.77, 90.48, 88.25, 77.88, 60.00])

# compute deterministic V0 from t0 prices and allocations
current_values = np.array([99.40, 98.39, 97.22, 92.79, 90.11, 86.60, 77.16, 60.00])  # from case handout

def current_portfolio_value(portfolio_dict):
    v0 = 0.0
    for r, amt in portfolio_dict.items():
        idx = rating_to_idx[r]
        v0 += amt * current_values[idx] / 100.0
    return v0

# Portfolio allocations (in EUR millions)
portfolio_I = {'AAA': 900, 'AA': 450, 'BBB': 150}
portfolio_II = {'BB': 900, 'B': 525, 'CCC': 75}

# Correlation values
rho_values = [0.0, 0.33, 0.66, 1.0]

# Cumulative probs and z-thresholds
cum_probs = np.cumsum(transition_matrix, axis=1)   # shape (7,8)

# Build thresholds so that low X => worse rating (D), high X => better (AAA)
# transition_matrix rows: starting rating AAA..CCC; columns: end rating AAA..D

cum_worst = np.cumsum(transition_matrix[:, ::-1], axis=1)     # columns now D..AAA
thr = stats.norm.ppf(cum_worst[:, :-1])                       # 7 finite thresholds per row

def map_X_to_rating_idx(X, thr_row):
    # thr_row length 7, increasing: (-inf,thr0]=D, ..., (thr6,inf)=AAA
    k = np.searchsorted(thr_row, X, side="right")             # k in {0,...,7} for D..AAA
    final_idx = 7 - k                                         # convert to AAA..D index (0..7)
    return final_idx


**Question 1**


In [72]:
# Simulation function for one-period, single-issuer-per-rating-class portfolio

def simulate_portfolio_single_issuer(portfolio_dict, rho, n_sims, seed=None):
    """
    One-period simulation, one issuer per rating class with positive exposure.
    Returns array of simulated portfolio values (in EUR millions).
    """
    rng = np.random.default_rng(seed)
    n_ratings_non_default = 7  # AAA..CCC

    # Exposure per starting rating
    amounts = np.zeros(n_ratings_non_default)
    for r, amt in portfolio_dict.items():
        idx = rating_to_idx[r]
        amounts[idx] = amt

    # Indices of ratings actually used
    issuer_indices = np.where(amounts > 0)[0]

    # Common systematic factor
    Z = rng.standard_normal(n_sims)
    sqrt_rho = np.sqrt(rho)
    sqrt_1mr = np.sqrt(1.0 - rho)

    portfolio_values = np.zeros(n_sims)

    for j in issuer_indices:
        # Idiosyncratic shocks
        eps = rng.standard_normal(n_sims)
        # Latent asset returns
        X = sqrt_rho * Z + sqrt_1mr * eps

        thr_row = thr[j]                          # j is starting rating index (AAA..CCC)
        final_idx = map_X_to_rating_idx(X, thr_row)
        prices = forward_values[final_idx]


        # Bond prices and contribution to portfolio value
        prices = forward_values[final_idx]   # size n_sims
        portfolio_values += amounts[j] * prices / 100.0

    return portfolio_values

def risk_measures(values, alpha, V0):
    """
    Compute VaR and ES of losses L = V0 - V1.
    Returns (V0, VaR_alpha, ES_alpha).
    """
    losses = V0 - values
    var = np.quantile(losses, alpha, method="higher")  # conservative, jump-safe
    es = losses[losses >= var].mean()
    return V0, var, es





In [73]:
# 1. Default Threshold Check 
#
BBB_idx = rating_to_idx['BBB']

# Analytic threshold from transition probability BBB->D
p_BBB_D = transition_matrix[BBB_idx, rating_to_idx['D']]
c_analytic = stats.norm.ppf(p_BBB_D)

# Threshold actually used by mapping (first cutoff = default)
c_used = thr[BBB_idx, 0]

print("BBB->D prob:", p_BBB_D)
print("Analytic cutoff:", c_analytic)
print("Cutoff used in simulation:", c_used)


# 2. Convergence Check

candidate_N = [500 , 1000 , 2000, 5000, 10000 , 50000, 100000 , 200000]

portfolio = portfolio_II
rho_check = 0.33
alpha = 0.995
V0_fixed = current_portfolio_value(portfolio)

print(f"{'N':>7} | {'VaR run1':>9} {'run2':>9} {'run3':>9} | {'range':>7}")
for n_sims in candidate_N:
    n_sims = int(n_sims)  # ensure integer

    vars_ = []
    for seed in (1, 42, 100):
        vals = simulate_portfolio_single_issuer(portfolio, rho=rho_check, n_sims=n_sims, seed=seed)
        _, var_alpha, es_alpha = risk_measures(vals, alpha=alpha, V0=V0_fixed)

        vars_.append(float(var_alpha))  # convert np.float64 -> float

    r = max(vars_) - min(vars_)
    print(f"{n_sims:7d} | {vars_[0]:9.2f} {vars_[1]:9.2f} {vars_[2]:9.2f} | {r:7.2f}")



BBB->D prob: 0.00168
Analytic cutoff: -2.9327262573342936
Cutoff used in simulation: -2.9327262573342936
      N |  VaR run1      run2      run3 |   range
    500 |    261.79    261.79    261.79 |    0.00
   1000 |    261.79    329.64    275.20 |   67.85
   2000 |    261.79    275.20    261.79 |   13.41
   5000 |    262.59    275.20    261.79 |   13.41
  10000 |    262.59    261.79    261.79 |    0.80
  50000 |    262.59    262.59    275.20 |   12.61
 100000 |    262.59    275.20    262.59 |   12.61
 200000 |    262.59    262.59    262.59 |    0.00


In [74]:
# === CONFIG ===
N_SIMS = 200000
ALPHAS = [0.90, 0.995]

portfolios = {
    "Portfolio I": portfolio_I,
    "Portfolio II": portfolio_II,
}

# === WRAPPER TO RUN ALL RHO VALUES FOR ONE PORTFOLIO ===

def compute_metrics_for_portfolio(portfolio_name, portfolio_dict, rho_values, n_sims, seed_base=100):
    results = []
    V0_actual = current_portfolio_value(portfolio_dict)  
    
    for k, rho in enumerate(rho_values):
        seed = seed_base + k
        vals = simulate_portfolio_single_issuer(portfolio_dict, rho, n_sims, seed)
        expected_value = vals.mean()
        
        row = {
            "portfolio": portfolio_name,
            "rho": rho,
            "V0_actual": V0_actual,      # Add this
            "ExpectedValue": expected_value,
        }
        
        for alpha in ALPHAS:
            EV, var_alpha, es_alpha = risk_measures(vals, alpha=alpha, V0=V0_actual)  # Pass V0
            row[f"VaR_{int(alpha*1000)/10:.1f}"] = var_alpha
            row[f"ES_{int(alpha*1000)/10:.1f}"] = es_alpha
        
        results.append(row)
    return results

# === RUN FOR BOTH PORTFOLIOS ===

all_results = []

all_results += compute_metrics_for_portfolio("Portfolio I", portfolio_I, rho_values, N_SIMS, seed_base=100)
all_results += compute_metrics_for_portfolio("Portfolio II", portfolio_II, rho_values, N_SIMS, seed_base=200)

# === PRINT IN SIMPLE TABLE-LIKE FORMAT ===

print("Single Issuer per Rating Class")
print("Portfolio | rho  | ExpVal  | VaR_90  | VaR_99.5 | ES_90   | ES_99.5")
for r in all_results:
    print(
        f"{r['portfolio']:10s} | "
        f"{r['rho']:<4.2f} | "
        f"{r['ExpectedValue']:7.2f} | "
        f"{r['VaR_90.0']:7.2f} | "
        f"{r['VaR_99.5']:8.2f} | "
        f"{r['ES_90.0']:7.2f} | "
        f"{r['ES_99.5']:8.2f}"
    )


Single Issuer per Rating Class
Portfolio | rho  | ExpVal  | VaR_90  | VaR_99.5 | ES_90   | ES_99.5
Portfolio I | 0.00 | 1476.48 |    5.37 |    27.86 |   11.50 |    54.74
Portfolio I | 0.33 | 1476.49 |    6.43 |    33.33 |   12.67 |    57.27
Portfolio I | 0.66 | 1476.50 |    5.37 |    36.76 |   13.97 |    65.81
Portfolio I | 1.00 | 1476.49 |   -1.41 |    48.93 |    0.49 |    83.56
Portfolio II | 0.00 | 1323.25 |   41.91 |   261.79 |   94.68 |   272.94
Portfolio II | 0.33 | 1323.15 |   41.91 |   275.20 |   99.83 |   339.07
Portfolio II | 0.66 | 1323.50 |   41.91 |   329.64 |  107.53 |   396.15
Portfolio II | 1.00 | 1323.14 |   41.91 |   423.51 |  126.23 |   423.51


**Question 2**

In [75]:
def simulate_portfolio_diversified(portfolio_dict, rho, n_sims, n_issuers_per_rating=100, seed=None):
    """
    One-period simulation, n_issuers_per_rating issuers per rating class.
    Exposure is split equally among issuers within each rating.
    Returns array of simulated portfolio values (in EUR millions).
    """
    rng = np.random.default_rng(seed)
    n_ratings_non_default = 7  # AAA..CCC

    # Exposure per starting rating
    amounts = np.zeros(n_ratings_non_default)
    for r, amt in portfolio_dict.items():
        idx = rating_to_idx[r]
        amounts[idx] = amt

    # Indices of ratings actually used
    rating_indices_used = np.where(amounts > 0)[0]

    # Common systematic factor (shared by all issuers in all ratings)
    Z = rng.standard_normal(n_sims)
    sqrt_rho = np.sqrt(rho)
    sqrt_1mr = np.sqrt(1.0 - rho)

    portfolio_values = np.zeros(n_sims)

    for j in rating_indices_used:
        # Exposure per issuer in this rating class
        exposure_per_issuer = amounts[j] / n_issuers_per_rating

        # Simulate all issuers in this rating class
        for issuer_k in range(n_issuers_per_rating):
            # Independent idiosyncratic shock for each issuer
            eps = rng.standard_normal(n_sims)
            X = sqrt_rho * Z + sqrt_1mr * eps

            thr_row = thr[j]                          # j is starting rating index (AAA..CCC)
            final_idx = map_X_to_rating_idx(X, thr_row)
            prices = forward_values[final_idx]


            # Bond prices and contribution to portfolio value
            prices = forward_values[final_idx]
            portfolio_values += exposure_per_issuer * prices / 100.0

    return portfolio_values


# === CONFIG ===
N_SIMS = 200000
N_ISSUERS = 100
ALPHAS = [0.90, 0.995]

# === RUN FOR BOTH PORTFOLIOS, ALL RHO ===

def compute_metrics_diversified(portfolio_name, portfolio_dict, rho_values, n_sims, n_issuers, seed_base=1000):
    V0 = current_portfolio_value(portfolio_dict)
    results = []
    
    for k, rho in enumerate(rho_values):
        seed = seed_base + k
        
        vals = simulate_portfolio_diversified(
            portfolio_dict=portfolio_dict,
            rho=rho,
            n_sims=n_sims,
            n_issuers_per_rating=n_issuers,
            seed=seed
        )
        
        expected_value = vals.mean()
        
        row = {
            "portfolio": portfolio_name,
            "rho": rho,
            "ExpectedValue": expected_value,
        }
        
        for alpha in ALPHAS:
            _, var_alpha, es_alpha = risk_measures(vals, alpha, V0)
            row[f"VaR_{int(alpha*1000)/10:.1f}"] = var_alpha
            row[f"ES_{int(alpha*1000)/10:.1f}"] = es_alpha
        
        results.append(row)
    
    return results


# === RUN ===

all_results_diversified = []

all_results_diversified += compute_metrics_diversified("Portfolio I", portfolio_I, rho_values, N_SIMS, N_ISSUERS, seed_base=1000)
all_results_diversified += compute_metrics_diversified("Portfolio II", portfolio_II, rho_values, N_SIMS, N_ISSUERS, seed_base=2000)

# === PRINT ===

print("\n100 Issuers per Rating Class")
print("Portfolio    | rho  | ExpVal  | VaR_90  | VaR_99.5 | ES_90   | ES_99.5")
for r in all_results_diversified:
    print(
        f"{r['portfolio']:12s} | "
        f"{r['rho']:<4.2f} | "
        f"{r['ExpectedValue']:7.2f} | "
        f"{r['VaR_90.0']:7.2f} | "
        f"{r['VaR_99.5']:8.2f} | "
        f"{r['ES_90.0']:7.2f} | "
        f"{r['ES_99.5']:8.2f}"
    )



100 Issuers per Rating Class
Portfolio    | rho  | ExpVal  | VaR_90  | VaR_99.5 | ES_90   | ES_99.5
Portfolio I  | 0.00 | 1476.50 |    0.77 |     2.18 |    1.24 |     2.57
Portfolio I  | 0.33 | 1476.51 |    3.12 |    16.34 |    7.09 |    23.43
Portfolio I  | 0.66 | 1476.51 |    3.48 |    30.16 |   11.10 |    50.06
Portfolio I  | 1.00 | 1476.50 |   -1.41 |    48.93 |    0.48 |    84.59
Portfolio II | 0.00 | 1323.43 |    5.72 |    12.45 |    8.16 |    14.33
Portfolio II | 0.33 | 1323.35 |   28.72 |   119.27 |   57.49 |   151.51
Portfolio II | 0.66 | 1323.32 |   33.40 |   236.91 |   95.24 |   292.21
Portfolio II | 1.00 | 1323.48 |   41.91 |   423.51 |  125.76 |   423.51


In [76]:
# === QUESTION 3.3: Tail Fatness Ratio ===

# Extract Portfolio II results for both cases
print("Portfolio II - Single Issuer:")
print("rho  | VaR_99.5 | ES_99.5  | Ratio")
for r in all_results:
    if r['portfolio'] == 'Portfolio II':
        ratio = r['ES_99.5'] / r['VaR_99.5']
        print(f"{r['rho']:.2f} | {r['VaR_99.5']:8.2f} | {r['ES_99.5']:8.2f} | {ratio:.4f}")

print("\nPortfolio II - 100 Issuers:")
print("rho  | VaR_99.5 | ES_99.5  | Ratio")
for r in all_results_diversified:
    if r['portfolio'] == 'Portfolio II':
        ratio = r['ES_99.5'] / r['VaR_99.5']
        print(f"{r['rho']:.2f} | {r['VaR_99.5']:8.2f} | {r['ES_99.5']:8.2f} | {ratio:.4f}")


Portfolio II - Single Issuer:
rho  | VaR_99.5 | ES_99.5  | Ratio
0.00 |   261.79 |   272.94 | 1.0426
0.33 |   275.20 |   339.07 | 1.2321
0.66 |   329.64 |   396.15 | 1.2018
1.00 |   423.51 |   423.51 | 1.0000

Portfolio II - 100 Issuers:
rho  | VaR_99.5 | ES_99.5  | Ratio
0.00 |    12.45 |    14.33 | 1.1514
0.33 |   119.27 |   151.51 | 1.2703
0.66 |   236.91 |   292.21 | 1.2334
1.00 |   423.51 |   423.51 | 1.0000
